In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, root_mean_squared_error

In [ ]:
df = pd.read_csv('df_depuracion.csv')

In [ ]:
X = df.drop(columns=['prices_per_night'])
y = df['prices_per_night']


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## Definimos el modelo y sus parámetros

In [ ]:
param_grid_reduced = {
    'n_estimators': [100, 500, 1000],
    'num_leaves': [31, 63, 127],
    'max_depth': [5, 7, 10],
    'learning_rate': [0.01, 0.05, 0.1],
    'min_child_samples': [20, 50],
    'subsample': [0.7, 0.8],
    'reg_lambda': [0, 1],
    'reg_alpha': [0, 1],
    'boosting_type': ['gbdt', 'dart'],
    'objective': ['regression']
}

In [ ]:
# Modelo base
lgb = LGBMRegressor(random_state=42)
# GridSearchCV
grid_search = GridSearchCV(
    estimator=lgb,
    param_grid=param_grid_reduced,
    scoring='neg_mean_squared_error',
    cv=5,  # Validación cruzada
    verbose=0
)
# Entrenamiento
grid_search.fit(X, y)  # Sustituye X e y por tus datos

In [ ]:
# Resultados
print("Mejores parámetros encontrados:")
print(grid_search.best_params_)

RESULTADO

·'boosting_type': 'gbdt'                                                                       

·'max_depth': 10

·'learning_rate': 0.05                                                                          

·'min_child_samples': 20

·'n_estimators': 1000                       

·'num_leaves': 63

·'objective': 'regression'                  

·'reg_alpha': 1

·'reg_lambda': 0                           

·'subsample': 0.7